# Day 8 Solution

In [1]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

In [3]:
input_data = open('input_data.txt', newline='\n').read()

### Problem #1

In [4]:
sample = """30373
25512
65332
33549
35390"""

sample = """30373
25512
65432
33649
35390"""

In [5]:
tree_grid_x = []

for row in input_data.splitlines():
# for row in sample.splitlines():
    tree_grid_y = []
    for col in row:
        tree_grid_y+=[int(col)]
    tree_grid_x+=[tree_grid_y]

In [6]:
tree_grid = np.array(tree_grid_x)

In [7]:
def is_visible(array1):
    largest_from_left = -1
    largest_from_right = -1
    lfl_array = []
    lfr_array = []
    for i in range(len(array1)):
        if array1[i]>largest_from_left:
            lfl_array.append(True)
            largest_from_left=array1[i]
        else:
            lfl_array.append(False)
    for i in range(len(array1)-1, 0-1, -1):
        if array1[i]>largest_from_right:
            lfr_array.append(True)
            largest_from_right=array1[i]
        else:
            lfr_array.append(False)
    lfr_array.reverse()

    result = np.logical_or(lfl_array, lfr_array)
    return result

In [8]:
visible_grid_horizontal = []

for row in tree_grid:
    visible_grid_horizontal.append([is_visible(row)])
visible_grid_horizontal = np.concatenate(visible_grid_horizontal, axis=0)

In [9]:
visible_grid_horizontal

array([[ True, False, False, ...,  True, False,  True],
       [ True, False,  True, ...,  True,  True,  True],
       [ True, False, False, ..., False,  True,  True],
       ...,
       [ True, False, False, ..., False,  True,  True],
       [ True, False, False, ...,  True, False,  True],
       [ True, False, False, ..., False, False,  True]])

In [10]:
visible_grid_vertical = []

for col in tree_grid.T:
    visible_grid_vertical.append([is_visible(col)])
visible_grid_vertical = np.concatenate(visible_grid_vertical, axis=0).T

In [11]:
visible_grid_vertical

array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False,  True, ...,  True,  True, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       [ True,  True,  True, ...,  True,  True,  True]])

In [12]:
final_result = np.logical_or(visible_grid_horizontal, visible_grid_vertical)

In [13]:
tree_grid

array([[2, 2, 0, ..., 1, 0, 0],
       [0, 0, 2, ..., 2, 1, 0],
       [2, 0, 0, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 1, 2, 0],
       [2, 1, 1, ..., 2, 1, 1],
       [2, 2, 2, ..., 0, 1, 2]])

In [14]:
final_result

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True,  True],
       [ True, False, False, ..., False,  True,  True],
       ...,
       [ True, False, False, ..., False,  True,  True],
       [ True, False, False, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [15]:
final_result.sum()

1789

### Problem #2

In [16]:
from itertools import product

In [17]:
class Tree:
    def __init__(self, coords, tree_grid):
        self.coords=coords
        self.x,self.y=self.coords
        self.tree_grid=tree_grid
        self.tree_height=tree_grid[self.x,self.y]
    
    @property
    def trees_to_left(self):
        trees = self.tree_grid[self.x]
        if len(trees)==0:
            return np.array([])
        return np.flip(trees[:self.y])
    
    @property
    def trees_to_right(self):
        trees = self.tree_grid[self.x]
        if len(trees)==0:
            return np.array([])
        return trees[self.y+1:]
    
    @property
    def trees_to_bottom(self):
        trees = self.tree_grid.T[self.y]
        if len(trees)==0:
            return np.array([])
        return np.flip(trees[:self.x])
    
    @property
    def trees_to_top(self):
        trees = self.tree_grid.T[self.y]
        if len(trees)==0:
            return np.array([])
        return trees[self.x+1:]
    
    @property
    def view_left(self):
        trees_to_left = self.trees_to_left
        if len(trees_to_left)==0:
            return 0
        for view_score,next_tree_height in enumerate(self.trees_to_left, 1):
            if next_tree_height >= self.tree_height:
                break
        return view_score
    
    @property
    def view_right(self):
        trees_to_right = self.trees_to_right
        if len(trees_to_right)==0:
            return 0
        for view_score,next_tree_height in enumerate(self.trees_to_right, 1):
            if next_tree_height >= self.tree_height:
                break
        return view_score
    
    @property
    def view_bottom(self):
        trees_to_bottom = self.trees_to_bottom
        if len(trees_to_bottom)==0:
            return 0
        for view_score,next_tree_height in enumerate(self.trees_to_bottom, 1):
            if next_tree_height >= self.tree_height:
                break
        return view_score
    @property
    def view_top(self):
        trees_to_top = self.trees_to_top
        if len(trees_to_top)==0:
            return 0
        for view_score,next_tree_height in enumerate(self.trees_to_top, 1):
            if next_tree_height >= self.tree_height:
                break
        return view_score
    
    @property
    def view_score(self):
        return self.view_left*self.view_right*self.view_top*self.view_bottom
        
    def __repr__(self):
        return f"{self.trees_to_left}\n{self.trees_to_right}\n{self.trees_to_bottom}\n{self.trees_to_top}"

In [18]:
trees = []
for x,y in list(product(range(len(tree_grid[0])),range(len(tree_grid[:,0])))):
    trees.append(Tree((x,y), tree_grid))

In [19]:
max([t.view_score for t in trees])

314820

Attempt 1 that failed:

In [93]:
tmp = is_visible(tree_grid[1])

In [94]:
def scenic_score(array1):
    largest_from_left = -1
    viewing_score_left = 0
    viewing_score_right = 0
    prev_val_left=-1
    prev_val_right=-1
    largest_from_right = -1
    lfl_array = []
    lfr_array = []
    for i in range(len(array1)):
        if prev_val_left==-1:
            viewing_score_left=0
        elif prev_val_left < array1[i]:
            viewing_score_left+=1
#             print(f"{prev_val_left} < {array1[i]} increasing viewing score left to {viewing_score_left}")
        else:
            viewing_score_left=1
#             print(f"{prev_val_left} is not < {array1[i]} setting viewing score left to {viewing_score_left}")
        prev_val_left=array1[i]
#         print(viewing_score_left)
        lfl_array.append(viewing_score_left)
    for i in range(len(array1)-1, 0-1, -1):
        if prev_val_right==-1:
            viewing_score_right=0
        elif prev_val_right < array1[i]:
            viewing_score_right+=1
#             print(f"{prev_val_right} < {array1[i]} increasing viewing score right to {viewing_score_right}")
        else:
            viewing_score_right=1
#             print(f"{prev_val_right} is not < {array1[i]} setting viewing score right to {viewing_score_right}")
        prev_val_right=array1[i]
        lfr_array.append(viewing_score_right)
#         print(viewing_score_right)
    lfr_array.reverse()
#     print(lfl_array, lfr_array)
    result = np.array(lfl_array)*np.array(lfr_array)
    return result

In [95]:
tree_grid

array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 4, 3, 2],
       [3, 3, 6, 4, 9],
       [3, 5, 3, 9, 0]])

In [96]:
visible_grid_horizontal = []

for row in tree_grid:
    visible_grid_horizontal.append([scenic_score(row)])
visible_grid_horizontal = np.concatenate(visible_grid_horizontal, axis=0)

In [97]:
visible_grid_horizontal

array([[0, 1, 2, 3, 0],
       [0, 1, 2, 1, 0],
       [0, 3, 2, 1, 0],
       [0, 1, 4, 1, 0],
       [0, 2, 1, 2, 0]])

In [98]:
visible_grid_vertical = []

for col in tree_grid.T:
    visible_grid_vertical.append([scenic_score(col)])
visible_grid_vertical = np.concatenate(visible_grid_vertical, axis=0).T

In [99]:
(visible_grid_horizontal*visible_grid_vertical).max()

8

In [111]:
test_array = np.array([3, 6, 4, 5, 3])

In [112]:
3 > test_array

array([False, False, False, False, False])

In [113]:
6>test_array

array([ True, False,  True,  True,  True])

In [108]:
tree_grid[:,2]

array([3, 5, 4, 6, 3])

In [101]:
visible_grid_horizontal.max(axis=0)

array([0, 3, 4, 3, 0])

In [102]:
visible_grid_vertical.max(axis=1)

array([0, 2, 4, 3, 0])

In [103]:
tree_grid[:20,70:80]

array([], shape=(5, 0), dtype=int64)

In [76]:
visible_grid_vertical[0:20,70:80]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 2,  1,  2,  2,  1,  1,  1,  2,  1,  1],
       [ 1,  2,  1,  1,  2,  2,  4,  1,  2,  4],
       [ 2,  6,  4,  2,  1,  6,  1,  6,  1,  1],
       [ 6,  1,  1,  1,  2,  1,  2,  2,  2,  2],
       [ 1,  6,  2,  2,  1,  4,  2,  1,  6,  6],
       [ 2,  2,  6,  6,  1,  1,  1,  4,  1,  1],
       [ 3,  1,  1,  1,  4,  4,  4,  1,  4,  2],
       [ 2,  1,  2,  1,  1,  1,  1,  4,  1,  2],
       [ 1,  2,  1,  6,  1,  2,  4,  1,  4,  1],
       [ 4,  2,  1,  2,  4,  9,  1,  4,  1,  2],
       [ 1,  1,  4,  1,  1,  2,  4,  1,  4,  3],
       [ 4,  4,  1,  2,  2,  1,  1,  1,  1,  2],
       [ 1,  1,  2,  3,  3,  4,  4,  4,  4,  1],
       [ 1,  2,  9, 16,  4,  1,  1,  1,  1,  2],
       [ 1,  1,  2,  3,  2,  2,  6,  2,  4,  9],
       [ 1,  1,  1,  2,  1,  6,  2,  1,  1,  2],
       [ 1,  1,  1,  1,  2,  1,  1,  2,  4,  1],
       [ 1,  1,  1,  1,  2,  2,  4,  1,  1,  4],
       [ 1,  2,  4,  4,  1,  1,  1,  2,  4,  1]])

In [73]:
(visible_grid_vertical*visible_grid_horizontal)[0:20,70:80]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  4,   1,   4,   4,   1,   1,   1,   4,   2,   1],
       [  1,   8,   1,   2,   4,   2,  16,   1,   8,   4],
       [  2,  12,  24,   2,   2,   6,   1,  36,   2,   1],
       [ 48,   1,   4,   1,   8,   1,   8,   2,   8,   2],
       [  1,  60,   8,   6,   2,   4,   8,   1,  36,  12],
       [  4,   2,  12,  18,   2,   1,   1,  16,   1,   2],
       [ 18,   1,   2,   1,  16,   4,  16,   1,   8,   6],
       [  8,   1,   4,   1,   2,   1,   2,  24,   1,   8],
       [  1,   8,   1,  24,   1,   4,  24,   1,  16,   1],
       [  4,   8,   1,   4,  12,  18,   1,  16,   1,   8],
       [  1,   2,  24,   1,   1,   4,  24,   1,   8,  18],
       [  8,   8,   1,   8,   2,   1,   3,   2,   1,   4],
       [  1,   1,   4,  18,   3,   4,   4,   8,   8,   1],
       [  1,   2,  18, 144,   8,   1,   4,   1,   2,   6],
       [  2,   1,   4,   9,   4,   2,  24,   2,   4,  36],
       [  2,   1,   1,   8,   1,  12,   2,   2,   1,   4

In [27]:
visible_grid_vertical.argmax(axis=0)

array([14, 41, 50, 67, 92, 21, 46, 25, 21, 78, 21, 39, 49, 46,  2,  5,  4,
       24, 30,  7, 49, 16, 72, 47, 79, 60,  2, 12, 29, 59, 20, 80,  4, 62,
       79, 61, 49, 34, 43, 88, 74, 18, 34, 59, 69, 29, 69, 70, 82, 23, 13,
       60, 68, 67, 91,  8, 30, 33,  4, 66, 27, 19, 26, 10, 79,  8, 11, 24,
       53, 50,  4, 87, 66, 14, 75, 10, 95, 27, 54, 95,  6, 75, 63, 73, 32,
       10, 33,  7, 36, 61, 15, 41, 66, 46,  4, 14, 41, 39, 20])

In [107]:
tree_grid

array([[3, 0, 3, 7, 3],
       [2, 5, 5, 1, 2],
       [6, 5, 4, 3, 2],
       [3, 3, 6, 4, 9],
       [3, 5, 3, 9, 0]])

In [105]:
visible_grid_horizontal

array([[0, 1, 2, 3, 0],
       [0, 1, 2, 1, 0],
       [0, 3, 2, 1, 0],
       [0, 1, 4, 1, 0],
       [0, 2, 1, 2, 0]])

In [106]:
visible_grid_vertical

array([[0, 0, 0, 0, 0],
       [1, 1, 2, 1, 1],
       [4, 2, 1, 2, 1],
       [1, 1, 2, 3, 2],
       [0, 0, 0, 0, 0]])

In [104]:
(visible_grid_horizontal*visible_grid_vertical)

array([[0, 0, 0, 0, 0],
       [0, 1, 4, 1, 0],
       [0, 6, 2, 2, 0],
       [0, 1, 8, 3, 0],
       [0, 0, 0, 0, 0]])

In [19]:
(visible_grid_horizontal*visible_grid_vertical).argmax(keepdims=True)

array([[1459]])